<a href="https://colab.research.google.com/github/Buse-cetin/topic_modelling_new/blob/main/LDA_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd 
import os

data = pd.read_csv('/content/drive/MyDrive/unwanted_clean.csv')


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80284 entries, 0 to 80283
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  80284 non-null  int64 
 1   tweet       80212 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


In [4]:
df=pd.DataFrame(data)

In [5]:
df.columns = df.columns.str.replace(' ', '')

In [8]:
reviews=data[['tweet']]
print(reviews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80284 entries, 0 to 80283
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   80212 non-null  object
dtypes: object(1)
memory usage: 627.3+ KB
None


In [9]:
reviews.sample(10)

,tweet
13261,akbil fiyatlarına zam yapmış gerekliyse yapılı...
20454,kadıköy osmanağa mahallesi serasker caddesi çi...
78856,gittiğin yer kör şişli istanbul yiğit ölür şan...
71535,akbilezam öğrenci arkadaşlarıma sesleniyorum z...
20717,ataşehir küçük bakkalköy mahallesi kayışdağı c...
55057,ongun ulasim istanbulil hayırdır bayram seyran...
1307,yalanını sevsinler işin gücün yalan fiko eko y...
29000,yürüyen merdiven asansör sorunu zaman çözülecek
56580,sn taksi dolmuş esnafı evine ekmek götüremiyor...
48684,tembelliğe alışmayın yürümek sağlıklıdır öğren...


In [10]:
reviews_text=reviews['tweet'].values.tolist()

In [11]:
import re
reviews_text=[re.sub(r'[^A-Za-z\s]','',text) for text in reviews_text] 

TypeError: ignored

In [12]:
from pprint import pprint
pprint(len(reviews_text[0]))

print(reviews_text[0:3])

137
['dakikadan fazladır taksi dksı görünen otobüsü bekliyoruz taksi reddetti istanbulun ulaşım sıkıntısından gerçekten yıldım taksi ibbşikayet', 'olimpiyat cadde üzeri pendik buraya araç park etmek yasak arkadaslar usulsüz şekilde park ediyorotobüs ler araç lar zaman zaman zorlanıyorgeçmekte ibbşikayet emniyet trafikşube', 'parklarda kontrol bakımları aksatarak canlarla oynuyorsunuz kadikoybelediyesi kadikoysikayet']


In [13]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(texts):
    for text in texts:
        yield(simple_preprocess(str(text),deacc=True))
        
data_words=list(sent_to_words(reviews_text))

In [14]:

import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')
new_stopwords = ["pkk","imamoğlu","akp","chp","terör","izmir","iki","ibb","ibbsikayet","istanbul","olimpiyat","belediye","blk","den","soylu","belediye"," "]
stopword.extend(new_stopwords)

def remove_stopwords(texts):
    return [[word for word in text if word not in stopword] for text in texts]

data_words_nostops=remove_stopwords(data_words)
    



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
import gensim

bigram=gensim.models.Phrases(data_words_nostops,min_count=5,threshold=100)
trigram=gensim.models.Phrases(bigram[data_words_nostops],threshold=100)

bigram_mod=gensim.models.phrases.Phraser(bigram)

def make_bigram(texts):
    return [bigram_mod[doc] for doc in texts]

data_words_nostops_bigrams=make_bigram(data_words_nostops)

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [16]:
import spacy
nlp=spacy.load("en_core_web_sm",disable=['parser','ner'])

def lemmatize(texts,allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out=[]
    for sent in texts:
        doc=nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        
    return texts_out

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [17]:
data_lemmatized=lemmatize(data_words_nostops_bigrams,allowed_postags=['NOUN','ADJ','VERB','ADV'] )

In [18]:
import gensim.corpora as corpora
id2word=corpora.Dictionary(data_lemmatized)
texts=data_lemmatized
corpus=[id2word.doc2bow(text) for text in texts]

In [19]:
print([[(id2word[id],freq) for (id,freq) in cp]for cp in corpus[:2]])

[[('gercekten', 1), ('yıldım', 1)], [('arkadaslar', 1), ('cadde', 1), ('emniyet', 1), ('pendik', 1), ('trafiksube', 1), ('uzeri', 1)]]


In [20]:
#Modelleme kısmı
lda_model=gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=9,random_state=100,update_every=1,
                                         chunksize=100,passes=10,alpha='auto',per_word_topics=True)


pprint(lda_model.print_topics())

doc_lda=lda_model[corpus]

[(0,
  '0.073*"bir" + 0.028*"nisantası" + 0.017*"insan" + 0.015*"size" + '
  '0.014*"maslak" + 0.007*"gtmecidiyekoy_kadıkoy" + 0.007*"gtsisli_merter" + '
  '0.005*"gecen" + 0.005*"florya" + 0.004*"iptal"'),
 (1,
  '0.022*"basaksehir" + 0.020*"kagıthane" + 0.017*"bo" + 0.017*"gitti" + '
  '0.016*"nisantası" + 0.008*"gec" + 0.007*"soguk" + 0.007*"tesekkurler" + '
  '0.006*"gtmecidiyekoy_kadıkoy" + 0.006*"gtsisli_merter"'),
 (2,
  '0.021*"nisantası" + 0.018*"avcılar" + 0.017*"bak" + 0.015*"olu" + '
  '0.011*"guzel" + 0.011*"egm" + 0.011*"topkapı" + 0.009*"isler" + '
  '0.007*"gtsisli_merter" + 0.007*"gtmecidiyekoy_kadıkoy"'),
 (3,
  '0.051*"olmaz" + 0.046*"taksim" + 0.015*"bagcılar" + 0.014*"olmus" + '
  '0.008*"gtsisli_merter" + 0.008*"gtmecidiyekoy_kadıkoy" + 0.008*"trafik" + '
  '0.006*"sultangazi_kagıthane" + 0.006*"kucukcekmece_florya" + 0.005*"doyar"'),
 (4,
  '0.044*"gelir" + 0.041*"sirinevler" + 0.022*"nisantası" + 0.021*"beylikduzu" '
  '+ 0.014*"biter" + 0.013*"devam" + 0.012*"h

In [21]:
print(list(doc_lda)[0])

([(0, 0.115375355), (1, 0.09476601), (2, 0.10766928), (3, 0.13625033), (4, 0.10957632), (5, 0.1069682), (6, 0.10093839), (7, 0.10821385), (8, 0.12024225)], [(0, [7]), (1, [6])], [(0, [(7, 0.9999941)]), (1, [(6, 0.99938625)])])


In [22]:
print(lda_model[corpus[0]])
print("*****************")
print("The Topics Distribution for first doc: ")
print(lda_model[corpus[0]][0])

([(0, 0.115375355), (1, 0.09476601), (2, 0.10766928), (3, 0.13625033), (4, 0.10957632), (5, 0.1069682), (6, 0.10093839), (7, 0.10821385), (8, 0.12024225)], [(0, [7]), (1, [6])], [(0, [(7, 0.9999941)]), (1, [(6, 0.99938625)])])
*****************
The Topics Distribution for first doc: 
[(0, 0.115375355), (1, 0.09476601), (2, 0.10766928), (3, 0.13625033), (4, 0.10957632), (5, 0.1069682), (6, 0.10093839), (7, 0.10821385), (8, 0.12024225)]


In [23]:

print("Perplexity: ",lda_model.log_perplexity(corpus))

from gensim.models import CoherenceModel
coherence_model_lda=CoherenceModel(model=lda_model,texts=data_lemmatized,dictionary=id2word,coherence='c_v')

coherence_lda=coherence_model_lda.get_coherence()
print('Coherence Score: ',coherence_lda)

Perplexity:  -12.820893884663217
Coherence Score:  0.693589900669548


In [24]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.2 MB/s eta 0:00:00


In [30]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)


pyLDAvis.enable_notebook()

vis=pyLDAvis.gensim_models.prepare(lda_model,corpus,id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.049699  0.054517       1        1  13.752096
8      0.012057 -0.014108       2        1  12.157453
0      0.016012  0.001319       3        1  11.683610
4      0.029804 -0.130520       4        1  11.075959
2      0.015450  0.008412       5        1  10.883791
5      0.015408  0.005780       6        1  10.813859
7      0.019942  0.064766       7        1  10.396786
6     -0.164729  0.001424       8        1   9.661951
1      0.006357  0.008411       9        1   9.574496, topic_info=                        Term         Freq        Total Category  logprob  \
2309                 besikta  1715.000000  1715.000000  Default  30.0000   
32                       bir  1800.000000  1800.000000  Default  29.0000   
14715                  sisli  1551.000000  1551.000000  Default  28.0000   
2066                   olmaz  1487.000000  1487.000000  Default  27.0000   
111                   taksim  1322.000000  1322.000000  Default  26.0000   
...                      ...          ...          ...      ...      ...   
12713              nisantası   328.010361  5529.224549   Topic9  -4.1159   
39856  gtmecidiyekoy_kadıkoy   129.463360  1359.007749   Topic9  -5.0455   
39857         gtsisli_merter   129.463360  1359.007749   Topic9  -5.0455   
38391   sultangazi_kagıthane    61.402756   790.380889   Topic9  -5.7915   
39635          etiler_levent    58.898202   629.255633   Topic9  -5.8331   

       loglift  
2309   30.0000  
32     29.0000  
14715  28.0000  
2066   27.0000  
111    26.0000  
...        ...  
12713  -0.4787  
39856  -0.0050  
39857  -0.0050  
38391  -0.2090  
39635  -0.0227  

[393 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
218       6  0.977054         acar
172       7  0.984280       adalet
2598      1  0.987193      adamlar
321       6  0.994734         adre
2521      9  0.987164         agır
...     ...       ...          ...
1336      6  0.997933          yıl
2180      3  0.980224        yılda
2300      1  0.985444       yıllık
504       4  0.996628        zaman
2116      6  0.998649  zeytinburnu

[530 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 9, 1, 5, 3, 6, 8, 7, 2])

In [29]:
for m, cv in zip(x,coherence_vals):
    print("num topics: ",m,'has coherence value of :',round(cv,4))

NameError: ignored